# Vorhersagen mit einem trainierten Modell treffen

In [1]:
import joblib

scaler = joblib.load("scaler.pkl")
clf = joblib.load("knn_model.pkl")

In [2]:
import numpy as np

try:
    clf.predict(np.array([1, 7, 8, 3, 4]))

except Exception as e:
    print(f"Fehler bei der Vorhersage:\n{e}")

Fehler bei der Vorhersage:
Expected 2D array, got 1D array instead:
array=[1 7 8 3 4].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.


In [3]:
import pandas as pd

feature_names = [
    "age", "sex", "cp", "trestbps", "chol", "fbs",
    "restecg", "thalach", "exang", "oldpeak", "slope",
    "ca", "thal"
]

neuer_partient = np.array([[57, 1, 2, 130, 236, 0, 0, 174, 0, 0.0, 1, 1, 2]])
neuer_partient_df = pd.DataFrame(neuer_partient, columns=feature_names)
neuer_partient_df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,57.0,1.0,2.0,130.0,236.0,0.0,0.0,174.0,0.0,0.0,1.0,1.0,2.0


In [4]:
neuer_partient_scaled = scaler.transform(neuer_partient_df)
neuer_partient_scaled

array([[ 0.27621823,  0.72250438,  0.98808912, -0.02140663, -0.20580267,
        -0.38330071, -1.04610909,  1.07074858, -0.67663234, -0.92086403,
        -0.69498803,  0.32186034, -0.50904773]])

In [5]:
prediction = clf.predict(neuer_partient_scaled)
print(f"Vorhersage (0 = keiner herzerkrankung, 1 = Herzerkrankung) {prediction}")

Vorhersage (0 = keiner herzerkrankung, 1 = Herzerkrankung) [0]


In [6]:
def add_partient(partient_data: list, df: pd.DataFrame = None) -> pd.DataFrame:

    feature_names = [
    "age", "sex", "cp", "trestbps", "chol", "fbs",
    "restecg", "thalach", "exang", "oldpeak", "slope",
    "ca", "thal"
    ]
    
    if len(partient_data) != len(feature_names):
        raise ValueError(f"Es werden {len(partient_data)} wurde übergeben.")
    
    new_entry = pd.DataFrame([partient_data], columns=feature_names)

    if df is None or df.empty:
        df_new = new_entry.copy()
    else:
        if list(df.columns) != feature_names:
            raise ValueError("Die Spalten im DataFrame stimmen nicht mit den erwartungen überein")
        df_new = pd.concat([df, new_entry], ignore_index=True)

    print("Neuer Partient wurde erfolgreich hinzugefügt")
    return df_new

In [7]:
X_neue_patienten = pd.DataFrame(columns=feature_names)

df_new_patients = add_partient([57, 1, 2, 130, 236, 0, 0, 174, 0, 0.0, 1, 1, 2], df=X_neue_patienten)
df_new_patients

Neuer Partient wurde erfolgreich hinzugefügt


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,57,1,2,130,236,0,0,174,0,0.0,1,1,2


In [8]:
new_patients = [
    [57, 1, 2, 130, 236, 0, 0, 174, 0, 0.0, 1, 1, 2],
    [63, 0, 1, 145, 233, 1, 2, 150, 0, 2.3, 0, 0, 1],
    [51, 1, 0, 120, 198, 0, 1, 160, 1, 1.4, 2, 1, 3]
]

for patient in new_patients:
    df_new_patients = add_partient(patient, df_new_patients)

df_new_patients

Neuer Partient wurde erfolgreich hinzugefügt
Neuer Partient wurde erfolgreich hinzugefügt
Neuer Partient wurde erfolgreich hinzugefügt


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,57,1,2,130,236,0,0,174,0,0.0,1,1,2
1,57,1,2,130,236,0,0,174,0,0.0,1,1,2
2,63,0,1,145,233,1,2,150,0,2.3,0,0,1
3,51,1,0,120,198,0,1,160,1,1.4,2,1,3


In [9]:
X_new_scaled = scaler.transform(df_new_patients)
predictions = clf.predict(X_new_scaled)
print(f"Vorhersagen (0 = keine Herzerkrankung, 1 = Herzerkrankung): {predictions}")

Vorhersagen (0 = keine Herzerkrankung, 1 = Herzerkrankung): [0 0 1 0]


In [10]:
clf.predict_proba(X_new_scaled)

array([[0.6, 0.4],
       [0.6, 0.4],
       [0.2, 0.8],
       [0.8, 0.2]])